In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN3_3 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 3000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
        self.power_mean = 0 
        self.power_std = 0 
        self.sum = 0 
        self.sum_deviation = 0 
        self.N = 0 
        
    def add_sample(self, sample): 
        self.N += 1 
        power = sample[0][0]
        self.sum += power 
        self.power_mean = self.sum / self.N 
        self.sum_deviation += (power - self.power_mean) ** 2
        self.power_std = np.sqrt(self.sum_deviation / self.N)
        
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    next_power_states = np.array([0 if val[3] is None else (val[3][0] - memory.power_mean) / memory.power_std  
                        for val in batch])
    
    states[:, 0] = (states[:, 0] - memory.power_mean) / memory.power_std 
    next_states[:, 0] = next_power_states
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {episode}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: 2
maximum steps, simulation is done ... 
Pre-training...Episode: 0
maximum steps, simulation is done ... 
Pre-training...Episode: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer

maximum steps, simulation is done ... 
Episode: 41 Total reward: -3378.3400316164066 Explore P: 0.3311 SOC: 1.0000 Cumulative_SOC_deviation: 328.3896 Fuel Consumption: 94.4436
maximum steps, simulation is done ... 
Episode: 42 Total reward: -2696.0910015932427 Explore P: 0.3224 SOC: 1.0000 Cumulative_SOC_deviation: 261.2268 Fuel Consumption: 83.8230
maximum steps, simulation is done ... 
Episode: 43 Total reward: -2718.998913982208 Explore P: 0.3140 SOC: 0.9992 Cumulative_SOC_deviation: 263.8167 Fuel Consumption: 80.8318
maximum steps, simulation is done ... 
Episode: 44 Total reward: -2825.8775612117074 Explore P: 0.3057 SOC: 1.0000 Cumulative_SOC_deviation: 274.4513 Fuel Consumption: 81.3646
maximum steps, simulation is done ... 
Episode: 45 Total reward: -2700.2626837889193 Explore P: 0.2977 SOC: 1.0000 Cumulative_SOC_deviation: 261.3659 Fuel Consumption: 86.6040
maximum steps, simulation is done ... 
Episode: 46 Total reward: -1878.4037938657211 Explore P: 0.2899 SOC: 0.9220 Cumula

maximum steps, simulation is done ... 
Episode: 88 Total reward: -345.737164693684 Explore P: 0.0983 SOC: 0.6422 Cumulative_SOC_deviation: 29.4177 Fuel Consumption: 51.5604
maximum steps, simulation is done ... 
Episode: 89 Total reward: -330.57058533843525 Explore P: 0.0960 SOC: 0.6423 Cumulative_SOC_deviation: 27.9543 Fuel Consumption: 51.0276
maximum steps, simulation is done ... 
Episode: 90 Total reward: -590.5346063772503 Explore P: 0.0936 SOC: 0.9491 Cumulative_SOC_deviation: 51.4629 Fuel Consumption: 75.9052
maximum steps, simulation is done ... 
Episode: 91 Total reward: -435.9450890655883 Explore P: 0.0914 SOC: 0.6367 Cumulative_SOC_deviation: 38.5152 Fuel Consumption: 50.7930
maximum steps, simulation is done ... 
Episode: 92 Total reward: -335.2124074703514 Explore P: 0.0892 SOC: 0.6345 Cumulative_SOC_deviation: 28.4942 Fuel Consumption: 50.2700
maximum steps, simulation is done ... 
Episode: 93 Total reward: -1492.3614932507892 Explore P: 0.0870 SOC: 1.0000 Cumulative_SOC_

maximum steps, simulation is done ... 
Episode: 135 Total reward: -238.76538621284826 Explore P: 0.0343 SOC: 0.6075 Cumulative_SOC_deviation: 18.9272 Fuel Consumption: 49.4929
maximum steps, simulation is done ... 
Episode: 136 Total reward: -221.8652741328727 Explore P: 0.0336 SOC: 0.6776 Cumulative_SOC_deviation: 16.8369 Fuel Consumption: 53.4958
maximum steps, simulation is done ... 
Episode: 137 Total reward: -167.82207082255465 Explore P: 0.0330 SOC: 0.6075 Cumulative_SOC_deviation: 11.9668 Fuel Consumption: 48.1537
maximum steps, simulation is done ... 
Episode: 138 Total reward: -170.8438270860071 Explore P: 0.0324 SOC: 0.6076 Cumulative_SOC_deviation: 12.2759 Fuel Consumption: 48.0853
maximum steps, simulation is done ... 
Episode: 139 Total reward: -147.19367330509894 Explore P: 0.0318 SOC: 0.6103 Cumulative_SOC_deviation: 9.9494 Fuel Consumption: 47.6992
maximum steps, simulation is done ... 
Episode: 140 Total reward: -154.99028116191968 Explore P: 0.0312 SOC: 0.6113 Cumulat

maximum steps, simulation is done ... 
Episode: 182 Total reward: -107.5809920195119 Explore P: 0.0167 SOC: 0.6059 Cumulative_SOC_deviation: 5.9711 Fuel Consumption: 47.8703
maximum steps, simulation is done ... 
Episode: 183 Total reward: -106.76106049748513 Explore P: 0.0165 SOC: 0.6078 Cumulative_SOC_deviation: 5.8412 Fuel Consumption: 48.3492
maximum steps, simulation is done ... 
Episode: 184 Total reward: -106.365695116277 Explore P: 0.0163 SOC: 0.6057 Cumulative_SOC_deviation: 5.7880 Fuel Consumption: 48.4861
maximum steps, simulation is done ... 
Episode: 185 Total reward: -108.4782858202962 Explore P: 0.0162 SOC: 0.6096 Cumulative_SOC_deviation: 6.0139 Fuel Consumption: 48.3395
maximum steps, simulation is done ... 
Episode: 186 Total reward: -110.58241062112232 Explore P: 0.0160 SOC: 0.6042 Cumulative_SOC_deviation: 6.2888 Fuel Consumption: 47.6943
maximum steps, simulation is done ... 
Episode: 187 Total reward: -103.85098490570293 Explore P: 0.0158 SOC: 0.6043 Cumulative_SO

In [11]:
with open("DDQN3_3_mass1200.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("DDQN3_3.pkl", "rb") as f: 
#     data = pickle.load(f)